In [1]:
#import packages and functions
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import os
import fiona
import shapely

In [2]:
from general_functions import *

# Delfland

In [3]:
#define relative paths
waterschap = 'Delfland'
gdb_path = "../../Data_preprocessed/Waterschappen/Delfland/Watersysteem.gdb"
output_gpkg_path = "../../Data_postprocessed/Waterschappen/Delfland"

In [4]:
Delfland = read_gpkg_layers(gpkg_path = gdb_path, 
                        variables = ['stuw', 
                                     'gemaal', 
                                     'watergang',
                                     'duikersifonhevel',
                                     'peilgebiedpraktijk',
                                     'keerschot']) 
                                     # 'peilafwijkinggebied', 
                                     # 'pomp'])
                                     # 'streefpeil'])
#change names
Delfland['hydroobject'] = Delfland.pop('watergang')


### Adjust column names

In [5]:
#discard irrelevant data of Delfland Delfland, and create a uniform dataset compared to the other waterschappen
#Stuw
Delfland['stuw'] = Delfland['stuw'][['CODE', 'GLOBALID', 'geometry']]
Delfland['stuw'] = Delfland['stuw'].rename(columns={'CODE': 'code', 'GLOBALID': 'globalid'})
Delfland['stuw']['nen3610id'] = 'dummy_nen3610id_stuw_' + Delfland['stuw'].index.astype(str)

#Gemaal

#determine aanvoer en afvoer gemalen
Delfland['gemaal']['func_aanvoer'], Delfland['gemaal']['func_afvoer'], Delfland['gemaal']['func_circulatie']  = False, False, False #default is False
Delfland['gemaal'].FUNCTIEGEMAAL_resolved.fillna(Delfland['gemaal'].WS_SOORTGEMAAL)#sometimes recircualtie is located in another column, but sometimes they are different. Only fill in for NaN
Delfland['gemaal']['FUNCTIEGEMAAL_resolved'] = Delfland['gemaal']['FUNCTIEGEMAAL_resolved'].astype(str) 

Delfland['gemaal'].loc[Delfland['gemaal'].FUNCTIEGEMAAL_resolved.str.contains('Onbekend|Onderbemaling|Afvoergemaal|Af-'), 'func_afvoer'] = True
Delfland['gemaal'].loc[Delfland['gemaal'].FUNCTIEGEMAAL_resolved.str.contains('Opmaling|Aanvoer'), 'func_aanvoer'] = True
Delfland['gemaal'].loc[Delfland['gemaal'].FUNCTIEGEMAAL_resolved.str.contains('Overig|circulatie'), 'func_circulatie'] = True
Delfland['gemaal'].loc[(Delfland['gemaal'].func_afvoer == False) &
                       (Delfland['gemaal'].func_aanvoer == False) &
                       (Delfland['gemaal'].func_circulatie == False), 'func_afvoer'] = True #set to afvoergemaal is there the function is unknown

Delfland['gemaal'] = Delfland['gemaal'][['GLOBALID', 'func_afvoer', 'func_aanvoer', 'func_circulatie', 'geometry']]
Delfland['gemaal'] = Delfland['gemaal'].rename(columns={'GLOBALID': 'globalid'})
Delfland['gemaal']['code'] = 'dummy_code_gemaal_' + Delfland['gemaal'].index.astype(str)
Delfland['gemaal']['nen3610id'] = 'dummy_nen3610id_gemaal_' + Delfland['gemaal'].index.astype(str)

#Hydroobject
Delfland['hydroobject'] = Delfland['hydroobject'][['GLOBALID', 'geometry']]
Delfland['hydroobject'] = Delfland['hydroobject'].rename(columns={'GLOBALID': 'globalid'})
Delfland['hydroobject']['code'] = 'dummy_code_hydroobject_' + Delfland['hydroobject'].index.astype(str)
Delfland['hydroobject']['nen3610id'] = 'dummy_nen3610id_hydroobject_' + Delfland['hydroobject'].index.astype(str)

#Keerschot
Delfland['keerschot'] = Delfland['keerschot'][['GLOBALID', 'geometry']]
Delfland['keerschot'] = Delfland['keerschot'].rename(columns={'GLOBALID': 'globalid'})
Delfland['keerschot']['code'] = 'dummy_code_keerschot_' + Delfland['keerschot'].index.astype(str)
Delfland['keerschot']['nen3610id'] = 'dummy_nen3610id_keerschot_' + Delfland['keerschot'].index.astype(str)

#duikersifonhevel
Delfland['duikersifonhevel'] = Delfland['duikersifonhevel'][['CODE', 'GLOBALID', 'geometry']]
Delfland['duikersifonhevel'] = Delfland['duikersifonhevel'].rename(columns={'CODE':'code', 'GLOBALID': 'globalid'})
Delfland['duikersifonhevel']['code'] = 'dummy_code_duikersifonhevel_' + Delfland['duikersifonhevel'].index.astype(str)
Delfland['duikersifonhevel']['nen3610id'] = 'dummy_nen3610id_duikersifonhevel_' + Delfland['duikersifonhevel'].index.astype(str)

#afsluitmiddel
#niet geleverd

#Peilgebiedpraktijk
Delfland['peilgebiedpraktijk'] = Delfland['peilgebiedpraktijk'][['WS_HOOGPEIL', 'CODE', 'GLOBALID', 'geometry']]
Delfland['peilgebiedpraktijk']['nen3610id'] = 'dummy_nen3610id_peilgebiedpraktijk_' + Delfland['peilgebiedpraktijk'].index.astype(str)
Delfland['peilgebiedpraktijk'] = Delfland['peilgebiedpraktijk'].rename(columns={'WS_HOOGPEIL': 'streefpeil', 'CODE':'code', 'GLOBALID':'globalid'})

#Streefpeil
Delfland['streefpeil'] = pd.DataFrame()
Delfland['streefpeil']['waterhoogte'] = Delfland['peilgebiedpraktijk']['streefpeil']
Delfland['streefpeil']['globalid'] = Delfland['peilgebiedpraktijk']['globalid']
Delfland['streefpeil']['geometry'] = None
Delfland['streefpeil'] = gpd.GeoDataFrame(Delfland['streefpeil'], geometry = 'geometry')

Delfland['peilgebied'] = Delfland['peilgebiedpraktijk']

### Add column to determine the HWS_BZM

In [6]:
Delfland['peilgebied']['HWS_BZM'] = False
Delfland['peilgebied'].loc[Delfland['peilgebied'].code == 'BZM 1', 'HWS_BZM'] = True #looked up manually

In [7]:
Delfland['streefpeil'].waterhoogte = Delfland['streefpeil'].waterhoogte.round(2)

In [8]:
#delete irrelvant data
variables = ['peilgebiedpraktijk']

for variable in variables:
    if str(variable) in Delfland:
        del Delfland[variable]

In [9]:
Delfland['peilgebied']

,streefpeil,code,globalid,geometry,nen3610id,HWS_BZM
0,-5.30,DOP 17,{2531CB3F-6001-4B96-8199-6C32F2A3F110},"MULTIPOLYGON (((90612.823 449598.605, 90683.17...",dummy_nen3610id_peilgebiedpraktijk_0,False
1,-0.92,DPP 2,{98322829-BD2C-4509-88EB-3020D4F5B7AE},"MULTIPOLYGON (((74942.850 449582.666, 74945.04...",dummy_nen3610id_peilgebiedpraktijk_1,False
2,-2.20,DPM 1 a,{CECA72F6-1CB0-4959-B074-16A25DDFFBCC},"MULTIPOLYGON (((77686.692 438546.781, 77761.73...",dummy_nen3610id_peilgebiedpraktijk_2,False
3,-5.40,DOP 18,{CBD8FADF-568E-4202-A96F-A1B9BD3128D3},"MULTIPOLYGON (((90779.645 449399.310, 90774.26...",dummy_nen3610id_peilgebiedpraktijk_3,False
4,-0.25,BZM 17,{25EC567D-B04A-4D3E-992B-42A752D7C707},"MULTIPOLYGON (((71961.494 444699.340, 71824.22...",dummy_nen3610id_peilgebiedpraktijk_4,False
...,...,...,...,...,...,...
1554,-2.50,KPN 2,{89BC3095-7743-419E-A3D5-97EC2E9358AB},"MULTIPOLYGON (((82221.600 445680.656, 82222.84...",dummy_nen3610id_peilgebiedpraktijk_1554,False
1555,-3.30,KLE 2 r,{532BBE4E-DB29-4A9D-8B1D-F82B1AC0E0E3},"MULTIPOLYGON (((80955.711 444823.816, 81352.20...",dummy_nen3610id_peilgebiedpraktijk_1555,False
1556,-2.61,KLE 2 m,{710B38F9-4FF0-43BC-89D4-2F2D2EA55561},"MULTIPOLYGON (((81042.188 444624.391, 80822.99...",dummy_nen3610id_peilgebiedpraktijk_1556,False
1557,NaN,KLE 2 n,{8FE7621C-AC97-4B16-BBF2-6CB0EDB21E19},"MULTIPOLYGON (((81312.894 444166.511, 81311.17...",dummy_nen3610id_peilgebiedpraktijk_1557,False


# Embed the boezems including the KRW-bodies

In [10]:
KRW = gpd.read_file(r"D:\Users\Bruijns\Documents\PR4750_20\Data_overig\KRW\Delfland\Opgedeelde_boezem_Delfland_definitief.shp")

In [11]:
KRW['code'] = 'dummy_code_BZM 1_' + KRW.index.astype(str)
KRW['nen3610id'] = 'owmnaam'
KRW['globalid'] = 'dummy_globalid_BZM_' + KRW.index.astype(str)
KRW['HWS_BZM'] = True
KRW['streefpeil'] = -0.43
KRW = KRW[['code', 'streefpeil', 'nen3610id', 'HWS_BZM', 'globalid', 'geometry']]
KRW = KRW.reset_index(drop=True)

In [12]:
#remove the previous boezem, and replace it for the new boezem which is splitted based on the KRW bodies
Delfland['peilgebied'] = Delfland['peilgebied'].loc[Delfland['peilgebied'].code != 'BZM 1']
Delfland['peilgebied'] = gpd.GeoDataFrame(pd.concat([Delfland['peilgebied'], KRW]), geometry='geometry')

Delfland['streefpeil'] = Delfland['peilgebied'][['streefpeil', 'globalid', 'geometry']]
Delfland['streefpeil'] = Delfland['streefpeil'].rename(columns={'streefpeil':'waterhoogte'})
Delfland['streefpeil']['geometry'] = np.nan

In [13]:
#add aggregation areas
aggregation_areas = gpd.read_file(r"D:\Users\Bruijns\Documents\PR4750_20\Data_preprocessed\Waterschappen\Delfland\Delfland_aanvoergebiedafvoergebied.gdb")
aggregation_areas['code'] =  aggregation_areas['CODE']
aggregation_areas['nen3610id'] = aggregation_areas['NAAM']
aggregation_areas['globalid'] = 'dummy_globalid_agg_area_' + aggregation_areas.index.astype(str)
aggregation_areas = aggregation_areas[['code', 'nen3610id', 'globalid', 'geometry']]

#remove old boezem, implement new one
aggregation_areas = aggregation_areas.loc[aggregation_areas.code != 'BZM']
aggregation_areas = pd.concat([aggregation_areas, KRW[['code', 'nen3610id', 'globalid', 'geometry']]])

Delfland['aggregation_area'] = gpd.GeoDataFrame(aggregation_areas, geometry='geometry')

### Check for the correct keys and columns

In [14]:
show_layers_and_columns(waterschap = Delfland)

stuw
['code' 'globalid' 'geometry' 'nen3610id']
type =  <class 'geopandas.geodataframe.GeoDataFrame'>
crs =  epsg:28992

gemaal
['globalid' 'func_afvoer' 'func_aanvoer' 'func_circulatie' 'geometry'
 'code' 'nen3610id']
type =  <class 'geopandas.geodataframe.GeoDataFrame'>
crs =  epsg:28992

duikersifonhevel
['code' 'globalid' 'geometry' 'nen3610id']
type =  <class 'geopandas.geodataframe.GeoDataFrame'>
crs =  epsg:28992

keerschot
['globalid' 'geometry' 'code' 'nen3610id']
type =  <class 'geopandas.geodataframe.GeoDataFrame'>
crs =  epsg:28992

hydroobject
['globalid' 'geometry' 'code' 'nen3610id']
type =  <class 'geopandas.geodataframe.GeoDataFrame'>
crs =  epsg:28992

streefpeil
['waterhoogte' 'globalid' 'geometry']
type =  <class 'geopandas.geodataframe.GeoDataFrame'>
crs =  epsg:28992

peilgebied
['streefpeil' 'code' 'globalid' 'geometry' 'nen3610id' 'HWS_BZM']
type =  <class 'geopandas.geodataframe.GeoDataFrame'>
crs =  epsg:28992

aggregation_area
['code' 'nen3610id' 'globalid' '

### Store data

In [15]:
# Check if the directory exists
if not os.path.exists(output_gpkg_path):
    # If it doesn't exist, create it
    os.makedirs(output_gpkg_path)
    
store_data(waterschap = Delfland, 
           output_gpkg_path = output_gpkg_path + '/Delfland')
